# Spatial submodule

This submodule features tools to extract subdomains around an origin point, as well as the ability to perform ordinary coordinate transforms on vectors.

In [ ]:
import osyris
import numpy as np
import matplotlib.pyplot as plt

au = osyris.units("au")

path = "osyrisdata/starformation"
data = osyris.Dataset(8, path=path).load()
ind = np.argmax(data["hydro"]["density"])
center = data["amr"]["position"][ind]

## Subdomain extraction

Extracting a subdomain returns a new dataset containing all cells, sink particles and particles **within** this subdomain. An 'origin' parameter specifies the coordinate from which the subdomain is extracted.

### Extracting a spherical subdomain

Here we extract a spherical subdomain of radius 1000 AU around the central cell.

In [ ]:
subdomain = osyris.extract_sphere(data, radius=1e3*au, origin=center)

In [ ]:
osyris.map(subdomain["hydro"]["density"], norm="log",
           dx=2000 * au, origin=center,
           direction="z")

### Extracting a cubic subdomain

Here, we extract all data within +/- 500 AU of the central cell in the x,y, and z directions.

In [ ]:
subdomain = osyris.extract_box(data,
                               xmin=-5e2*au, xmax=5e2*au,
                               ymin=-5e2*au, ymax=5e2*au,
                               zmin=-5e2*au, zmax=5e2*au,
                               origin=center)

osyris.map(subdomain["hydro"]["density"], norm="log",
           dx=2000 * au, origin=center,
           direction="z")

## Ordinary coordinate transforms

### Changing origins

We can simply change the origin of our dataset with

In [ ]:
osyris.translate(data, new_origin=center)

This translates all **length vectors** in the dataset by -1*new_origin.

### Changing basis

We can rotate datasets (or subdomains) using the **rotate** function. 'new_basis' is a vector specifying the z axis of the new geometric basis (eg. new_basis = [1,0,0] means the new z axis is oriented along the x axis of the original grid). Basis can also be either "top" and "side", meaning the new z axis can be aligned with the angular momentum vector, or perpendicular to it. This is particularly useful to compute azimuthal velocities for disks for example. Note however that you must pass the radius in which osyris will compute the angular momentum vector (dr_L).

In [ ]:
osyris.rotate(data, new_basis = "top", dr_L = 1e3*au)

### Spherical & Cylindrical coordinates

Spherical/cylindrical coordinates can be computed on Vectors simply by doing

In [ ]:
print(data["amr"]["position"].r) # spherical radius
print(data["amr"]["position"].cyl_r) # cylindrical radius
print(data["amr"]["position"].theta) # colatitude
print(data["amr"]["position"].phi) # azimuth

When doing these transformations, osyris checks wheter or not the units of the vector have the dimensions of a length, and if so, it returns $r$ (radius), $\theta$ (colatitude), $\phi$ (azimuth), or $r_{cyl}$ (cylindrical radius):
\begin{equation*}
r = \sqrt{x^2 + y^2 + z^2}
\end{equation*}

\begin{equation*}
\theta = arcos(\frac{z}{r})
\end{equation*}

\begin{equation*}
\phi = arctan(\frac{y}{x})
\end{equation*}

\begin{equation*}
r_{cyl} = \sqrt{x^2 + y^2}
\end{equation*}

If not, osyris will compute the spherical/cylindrical **components** of the vector using a rotation matrix. This allows it to convert any arbitrary cartesian vector $F_{x,y,z}$ into its spherical counterpart $F_{r,\theta,\phi}$ through

\begin{equation*}
\begin{pmatrix}
F_{r}\\
F_{\theta}\\
F_{\phi}
\end{pmatrix}
=
\begin{pmatrix}
sin(\theta)cos(\phi) & sin(\theta)sin(\phi) & cos(\theta)\\
cos(\theta)cos(\phi) & cos(\theta)sin(\phi) & -sin(\theta)\\
-sin(\phi) & cos(\phi) & 0
\end{pmatrix}
\begin{pmatrix}
F_{x}\\
F_{y}\\
F_{z}
\end{pmatrix}
\end{equation*}

and into its cylindrical counterpart $F_{r,\phi,z}$ through
\begin{equation*}
\begin{pmatrix}
F_{r}\\
F_{\phi}\\
F_{z}
\end{pmatrix}
=
\begin{pmatrix}
cos(\phi) & sin(\phi) & 0\\
-sin(\phi) & cos(\phi) & 0\\
0 & 0 & 1
\end{pmatrix}
\begin{pmatrix}
F_{x}\\
F_{y}\\
F_{z}
\end{pmatrix}
\end{equation*}

In [ ]:
print(data["hydro"]["velocity"].r) # radial velocity
print(data["hydro"]["velocity"].cyl_r) # cylindrical radial velocity (vr measured on x-y plane)
print(data["hydro"]["velocity"].theta) # meridonial velocity
print(data["hydro"]["velocity"].phi) # azimuthal velocity

In [ ]:
# Create figure
fig, ax = plt.subplots(2, 2, figsize=(12, 9))

osyris.histogram2d(data["amr"]["position"].r.to("au"), data["hydro"]["velocity"].r.to("km/s"),
                   norm="log", logx=True, cmap="viridis", ax=ax[0][0])
osyris.histogram2d(data["amr"]["position"].r.to("au"), data["hydro"]["B_field"].r.to("G"),
                   norm="log", logx=True, cmap="viridis", ax=ax[0][1])
osyris.histogram2d(data["amr"]["position"].r.to("au"), data["hydro"]["velocity"].phi.to("km/s"),
                   norm="log", logx=True, cmap="viridis", ax=ax[1][0])
osyris.histogram2d(data["amr"]["position"].r.to("au"), data["hydro"]["B_field"].theta.to("G"),
                   norm="log", logx=True, cmap="viridis", ax=ax[1][1])

ax[0][0].set_title("Radial velocity")
ax[0][1].set_title("Radial B field")
ax[1][0].set_title("Azimuthal velocity")
ax[1][1].set_title("Poloidal B field")

fig.tight_layout()